In [30]:
import os, re
from scripts.tools.issuetracker import IssuetrackerAPI
from datetime import date

In [36]:
rs_match  = re.compile(r"^\>?([\+\*]{2})([FU]RS\d+(?:\.\d+)?)([\+\*]{2})(.*)$").match
frs_match  = re.compile(r"^\>?([\+\*]{2})(FRS\d+(?:\.\d+)?)([\+\*]{2})(.*)$").match
header_match = re.compile(r"^\>?([\+\*]{2})([\w]+[^\:]+)(\:)?([\+\*]{2})(\:)?(.*)$").match

In [37]:
cache = None
api = None
sync_date = None
def _download():
    global cache, api, sync_date
    if api is None:
        api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    cache = api.download_issues(project_id="pbssoftware")
    sync_date = date.today()
    
def download(force=False):
    global cache, sync_date
    reason = None
    if cache is None:
        reason = "cache is None"
    elif force:
        reason = "forced re-cache"
    elif sync_date is None:
        reason = "can't determine last sync date"
    elif date.today() != sync_date:
        reason = "cache age > 0 days"
    if reason:
        print("Downloading issues:", reason)
        _download()
        filter_issues()
        
skip = {
    3057,  # 3.0 Web FRS
    3066,  # 3.0 Web FRS
    3194,  # URS
}
def relevant(i):
    if i.id in skip:
        return False
    if i.sprint_milestone != "Legacy" or i.tracker != "Specification":
        return False
    if i.status in ("Rejected", "Closed"):
        return False
    return True

def filter_issues():
    global cache, issues_30
    issues_30 = [i for i in cache.values() if relevant(i)]
    print("Filtered %d relevant issues" % len(issues_30))


def get_issues(recache=False):
    download(recache)
    return issues_30.copy()

In [38]:
class ListIter():
    def __init__(self, it):
        self.it = list(it)
        self.i = 0
    def get(self):
        try:
            v = self.it[self.i]
        except IndexError:
            v = None
        self.i += 1
        return v
    
    def unget(self):
        self.i -= 1
        if self.i < 0:
            self.i = 0
    

def extract_descr(d):
    it = ListIter(d.strip().splitlines())
    frs = []
    m = None
    while not m:
        line = it.get()
        if line is None:
            return frs
        m = toplevel_match(line)
        if m:
            while True:
                frs.append(line)
                line = next(it, None)
                if line is None:
                    break
                if header_match(line):
                    break
    return frs


def extract_descr(d):
    lines = ListIter(d.strip().splitlines())
    urs = []
    while True:
        l = lines.get()
        if l is None:
            break
        if rs_match(l):
            urs.append(l)
    return urs

def write_frs(f, frs):
    for i, lines in frs:
        tl = lines[0]
        f.write(tl + " %s (#%d)\n" % (i.subject, i.id))
        for line in lines[1:]: f.write(line+"\n")
        f.write("\n\n")
        
def write_frs2(f, frs):
    w = f.write
    for i, lines in frs:
        in_pre = False
        w("*+Issue #%d+*: %s\n" % (i.id, i.subject))
        for line in lines:
            if in_pre:
                prefix =""
            else:
                prefix = ">"
            if "<pre>" in line:
                in_pre = True
            if "</pre>" in line:
                in_pre = False
            w(prefix+line+"\n")
        w("\n\n")
    
def extract_frs():
    frs = []
    for i in get_issues():
        lines = extract_descr(i.description)
        if not lines: continue
        frs.append((i, lines))
    frs.sort(key=lambda v:v[0].id)
    return frs
        

def write_frs_numbers(fn): 
    """
    Verify the script found all FRS lines for the compiled
    FRS, by first extracting all possible lines directly from
    issue descriptions, then scanning the extracted FRS lines. 
    """

    frs = extract_frs()
    with open(fn, "w") as f:
        write_frs2(f, frs)
    

                
                

In [40]:
fn = "full_frs.txt"
write_frs_numbers(fn)
os.startfile(fn)